# Исследование объявлений о продаже квартир

1. В вашем распоряжении данные сервиса недвижимости — архив объявлений о продаже квартир в Санкт-Петербурге и соседних населённых пунктов за несколько лет. Архив представляет собой таблицу из 22 столбцов и 23699 строк:
* airports_nearest — расстояние до ближайшего аэропорта в метрах (м)
* balcony — число балконов
* ceiling_height — высота потолков (м)
* cityCenters_nearest — расстояние до центра города (м)
* days_exposition — сколько дней было размещено объявление (от публикации до снятия)
* first_day_exposition — дата публикации
* floor — этаж
* floors_total — всего этажей в доме
* is_apartment — апартаменты (булев тип)
* kitchen_area — площадь кухни в квадратных метрах (м²)
* last_price — цена на момент снятия с публикации
* living_area — жилая площадь в квадратных метрах(м²)
* locality_name — название населённого пункта
* open_plan — свободная планировка (булев тип)
* parks_around3000 — число парков в радиусе 3 км
* parks_nearest — расстояние до ближайшего парка (м)
* ponds_around3000 — число водоёмов в радиусе 3 км
* ponds_nearest — расстояние до ближайшего водоёма (м)
* rooms — число комнат
* studio — квартира-студия (булев тип)
* total_area — площадь квартиры в квадратных метрах (м²)
* total_images — число фотографий квартиры в объявлении

2. Наша задача — установить параметры недвижимости агломерации Санкт-Петербурга и научиться определять рыночную стоимость объектов недвижимости. Это позволит построить автоматизированную систему: она отследит аномалии и мошенническую деятельность. 

3. Плпн работы:
* Изучим основные параметры недвижимости: площадь, цена, число комнат, высота потолков.
* Cколько обычно занимает продажа. 
* Когда можно считать, что продажи прошли очень быстро, а когда необычно долго?
* Какие факторы больше всего влияют на стоимость квартиры? 
* Зависит ли цена от площади, числа комнат, удалённости от центра? 
* Есть ли зависимость цены от того, на каком этаже расположена квартира: первом, последнем или другом? 
* Есть ли зависимость от даты размещения: дня недели, месяца и года?
* 10 населённых пунктов с наибольшим числом объявлений. Cредняя цена квадратного метра в этих населённых пунктах? Населённые пункты с самой высокой и низкой стоимостью жилья? 
* Выясним, какая область входит в центр Санкт-Петербурга. Построим график зависимости как цена зависит от удалённости от центра. 
* Проанализируем следующие параметры квартир в центре: площадь, цена, число комнат, высота потолков. Сравним эти параметры с вцелом по агломерации.

## Часть 1. Изучение общей информации:
 1. [Изученеие файлов с данными, получение общей информации, загрузка библиотек](#1-bullet)


## Часть 2. Подготовка данных:
 1. [Нахождение пропущенных значений](#2-bullet)
 2. [Обработка пропущенных значений](#3-bullet)
 3. [Приведение данных к нужным типам](#4-bullet)
 4. [Внесение в таблицу расчетных значений](#5-bullet)
 
## Часть 3. Исследовательский анализ данных недвижимости Санкт-Петербурга:
 1. [Изучение параметров: площадь, цена, число комнат, высота потолков](#6-bullet)
 2. [Изучение продажи квартир: время продажи, среднее, медиана. Когда можно считать что продажи прошли очень быстро, когда необычно долго?](#7-bullet)
 3. [Изучение факторов которые больше всего влияют на стоимость квартиры](#8-bullet)
 4. [Анализ стоимости недвижимости 10-ти основных населенных пунктов агломерации](#9-bullet)
 5. [Исследование на тему: какая область Санкт-Петербурга входит в понятие центр](#10-bullet)
 6. [Анализ сегмента квартир в центре Санкт-Петербурга](#11-bullet)

## [Общий вывод по проведенному исследованию](#12-bullet)

### 1. Изучение общей информации:
#### 1. Изученеие файлов с данными, получение общей информации, загрузка библиотек<a id='1-bullet'></a>

In [1]:
# Импортируем библиотеки
import pandas as pd
import numpy as np
from IPython.display import display
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
def info(data):
    """
    Функция пробегает по признакам датафрейма и если тип у признака object, 
    то смотрит уникальные значения, в противном случае оценивается распределение
    """
    for column in data.columns:
        if data[column].dtype == 'object':
            print(column)
            print(data[column].unique())
            print()
            print(data[column].describe())
            print()
            
        else:
            
            print(column)
            import plotly.express as px
            fig = px.histogram(data, x = column, marginal = 'box', title = 'Распределение '+ column)
            fig.show()

In [3]:
# Проверяем общее количество пропусков по таблице в % выражении
def number_of_passes(data):
    """
    Функция пробегает по признакам датафрейма и высчитывает количество пропусков по таблице в % выражении
    """
    passes = pd.DataFrame(data.isnull().sum())
    passes['index'] = passes.index
    passes = passes.reset_index(drop =True)
    passes.columns = ['sum_NaN', 'specifications']
    passes = passes[['specifications', 'sum_NaN']]
    passes = passes.query('sum_NaN != 0').sort_values('sum_NaN').reset_index(drop =True)
    passes['passes_NaN'] = round(passes['sum_NaN'] / data.shape[0] * 100, 1)
    return passes

In [4]:
# Выгружаем данные из файла
data = pd.read_csv("C:/Users/vyugo/Documents/!Python/2. Проекты Я.Практикум/2. Недвижимость в СПБ/real_estate_data.csv", sep="\t")
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23699 entries, 0 to 23698
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   total_images          23699 non-null  int64  
 1   last_price            23699 non-null  float64
 2   total_area            23699 non-null  float64
 3   first_day_exposition  23699 non-null  object 
 4   rooms                 23699 non-null  int64  
 5   ceiling_height        14504 non-null  float64
 6   floors_total          23613 non-null  float64
 7   living_area           21796 non-null  float64
 8   floor                 23699 non-null  int64  
 9   is_apartment          2775 non-null   object 
 10  studio                23699 non-null  bool   
 11  open_plan             23699 non-null  bool   
 12  kitchen_area          21421 non-null  float64
 13  balcony               12180 non-null  float64
 14  locality_name         23650 non-null  object 
 15  airports_nearest   

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.0,2019-03-07T00:00:00,3,2.70,16.0,51.0,8,NaN,False,False,25.0,NaN,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
1,7,3350000.0,40.4,2018-12-04T00:00:00,1,NaN,11.0,18.6,1,NaN,False,False,11.0,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,10,5196000.0,56.0,2015-08-20T00:00:00,2,NaN,5.0,34.3,4,NaN,False,False,8.3,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0
3,0,64900000.0,159.0,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,NaN,False,False,NaN,0.0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0
4,2,10000000.0,100.0,2018-06-19T00:00:00,2,3.03,14.0,32.0,13,NaN,False,False,41.0,NaN,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0


#### Вывод
* Пропуски в данных образовались по всейвидимости по 2-м причинам: 1. Собственник недвижимости не полностью заполнил форму объявления. 2. Автоматизированная система по определению расстояний не смогла заполнить эти ячейки возможно из-за некорректно веденых собствеников данных адреса.

### 2. Подготовка данных:
#### 1. Нахождение пропущенных значений<a id='2-bullet'></a>

In [5]:
# Исправляем наименования столбцов 
data.columns = (['total_images', 'last_price', 'total_area', 'first_day_exposition',
       'rooms', 'ceiling_height', 'floors_total', 'living_area', 'floor',
       'is_apartment', 'studio', 'open_plan', 'kitchen_area', 'balcony',
       'locality_name', 'airports_nearest', 'city_centers_nearest',
       'parks_around_3000', 'parks_nearest', 'ponds_around_3000',
       'ponds_nearest', 'days_exposition'])

In [6]:
# Проверяем общее количество пропусков по таблице в % выражении
number_of_passes(data)

,specifications,sum_NaN,passes_NaN
0,locality_name,49,0.2
1,floors_total,86,0.4
2,living_area,1903,8.0
3,kitchen_area,2278,9.6
4,days_exposition,3181,13.4
5,parks_around_3000,5518,23.3
6,ponds_around_3000,5518,23.3
7,city_centers_nearest,5519,23.3
8,airports_nearest,5542,23.4
9,ceiling_height,9195,38.8


#### 2. Обработка пропущенных значений<a id='3-bullet'></a>

В результате анализа в таблице обнаужены столбцы с пропусками типа NaN в виду отсутствия обратной связи с заказчиком принимаем следующие решения по их устранению:

1. locality_name         0.2 % принимаем за "No_name" по причине невозможности установить населенный пункт.
2. floors_total          0.4 % принимаем что этаж расположения недвижимости является последним, судя по анализу этажность домов может быть любой, количество подобных замен 0,4% не повлияют на итоговую статистику.
3. living_area           8.0 % меняем на "медиану" для аналогичной площади квартиры т.к. все относительно стандартизировано.
4. kitchen_area          9.6 % меняем на "медиану" для аналогичной площади квартиры т.к. все относительно стандартизировано.
5. days_exposition      13.4 % меняем на "медиану" для аналогичной площади квартиры т.к. все относительно стандартизировано.
6. parks_around_3000    23.3 % оставляем NaN в количестве 5518 шт. по причине отсутствия информации.
7. ponds_around_3000    23.3 % оставляем NaN в количестве 5518 шт. по причине отсутствия информации.
8. city_centers_nearest 23.3 % оставляем NaN в количестве 5519 шт. по причине отсутствия информации.
9. airports_nearest     23.4 % оставляем NaN в количестве 14589 шт. по причине отсутствия информации.
10. ceiling_height       38.8 % меняем на "медиану" т.к. собственники скорее всего указали бы выдающиеся значения этого параметра.
11. balcony             48.6 % принимаем за 0 т.к. собственники скорее всего неуказали этот параметр по причине его отсутствия, наличие балкона это значительная переменная увеличивающая цену недвижимости.
12. ponds_nearest       61.6 % оставляем NaN в количестве 15620 шт. по причине отсутствия информации.
13. parks_nearest       65.9 % оставляем NaN в количестве 14589 шт. по причине отсутствия информации.
14. is_apartment        88.3 % т.к. из 12% указавших тип жилья - 2% апартаменты, считаем что собственники четко понимающие тип своей недвижимости указали эту важную особенность, остальные незаполнили т.к. незнают что это такое. Принимаем пропущенные значения за False.  
15. В столбце'rooms' исправили в 197 строках количество комнат с O-ми значениями на соответствующие по площади, ошибка возникла скорее всего по причине человеческого фактора, т.к. площади квартир относительно комнат стандартизированы - заполним медианным значением + 20% - чтобы захватить максимум соответствующего диапазона.

In [7]:
# 1.locality_name', заменяем NaN на "No_name", а также приведем к одному типу пояснения населенных пунктов
data['locality_name'] = data['locality_name'].fillna('No_name')
data['locality_name'] = data['locality_name'].str.replace('посёлок', 'поселок')
data['locality_name'] = data['locality_name'].str.replace('городской поселок', 'поселок городского типа')

In [8]:
# 2.'floors_total', приравниваем NaN к этажу расположения квартиры, т.к. этажность в агломерации представлена всеми этажами,
# а число пропусков всего 0,4% что неокажет значительного влияния на исследование
#data['floors_total'].value_counts().index.sort_values() # распространеная этажность в агломерации
data['floors_total'] = data['floors_total'].fillna(data['floor'])

In [9]:
# 3.'living_area' меняем NaN на медиану значений для квартир подобной многокомнатности т.к. квартиры стандартизованы
data['living_area'] = data.groupby('rooms')['living_area'].apply(lambda x: x.fillna(x.median()))
# Проверяем выполнение замен
#data['living_area'].isnull().sum()

In [10]:
# 4.'kitchen_area' меняем NaN на медиану значений для квартир подобной площади т.к. квартиры стандартизованы
data['kitchen_area'] = data.groupby('rooms')['kitchen_area'].apply(lambda x: x.fillna(x.median()))
# Проверяем выполнение замен
#display(data['kitchen_area'].isnull().sum())

In [11]:
# 10.'ceiling_height' меняем NaN на медиану значений для квартир подобной площади т.к. квартиры стандартизованы
#display(data.groupby('total_area')['ceiling_height'].apply(lambda x: x.fillna(x.median())))
data['ceiling_height'] = data.groupby('rooms')['ceiling_height'].apply(lambda x: x.fillna(x.median()))
# Проверяем выполнение замен
#data['ceiling_height'].isnull().sum()

In [12]:
# 11.'balcony' меняем NaN на 0
data['balcony'] = data['balcony'].fillna(0)

In [13]:
# 14.В виду того что уточнить данных нет возможности принимаем пропущенные значения не апартаментами
#data['is_apartment'].value_counts()
data['is_apartment'] = data['is_apartment'].fillna(False)
#display(data['is_apartment'])

In [14]:
# 3.'days_exposition' меняем NaN на медиану значений для квартир подобной многокомнатности т.к. квартиры стандартизованы
data['days_exposition'] = data.groupby('rooms')['days_exposition'].apply(lambda x: x.fillna(x.median()))
# Проверяем выполнение замен
#data['days_exposition'].isnull().sum()

In [15]:
# 15. В столбце'rooms' исправим в 197 строках количество комнат с O-ми значениями на соответствующие по площади
#display(data['rooms'].value_counts())
# Выясним медиану площади квартир относительно количества комнат, добавляем 20% чтобы охватить максимум диапазона
#display((data.groupby('rooms')['total_area'].median() * 1.2).head())
# Заполним 0-е значения столбца 'rooms' расчетными. 
data.loc[(data['rooms'] == 0) & (data['total_area'] <= 43.9), 'rooms'] = 1 
data.loc[(data['rooms'] == 0) & (data['total_area'] > 43.9) & (data['total_area'] <= 63.6), 'rooms'] = 2  
data.loc[(data['rooms'] == 0) & (data['total_area'] > 63.6) & (data['total_area'] <= 87.6), 'rooms'] = 3
data.loc[(data['rooms'] == 0) & (data['total_area'] > 87.6) & (data['total_area'] <= 119.2), 'rooms'] = 4
data.loc[(data['rooms'] == 0) & (data['total_area'] > 119.2) & (data['total_area'] <= 175.7), 'rooms'] = 5

#data.info()

#### 3. Приведение данных к нужным типам<a id='4-bullet'></a>

In [16]:
# 1.'first_day_exposition' float64 преобразуем в формат datetime64 для последующей обработки
data['first_day_exposition'] = pd.to_datetime(data['first_day_exposition'], format='%Y-%m-%dT%H:%M:%S')
data['first_day_exposition'] = data['first_day_exposition'].dt.round('d')
                                
# 2.'floors_total' float64 преобразуем в формат Int64, этажи не могут бытьдробным числом 
data['floors_total'] = data['floors_total'].astype('Int64')

# 3.'balcony' float64 преобразуем в формат Int64, количество балконов не может быть дробным числом
data['balcony'] = data['balcony'].astype('Int64')

# 4.'airports_nearest' float64 преобразуем в формат Int64, расстояние до аэропорта кратно метру
data['airports_nearest'] = data['airports_nearest'].astype('Int64')

# 5.'parks_around_3000' float64 преобразуем в формат Int64, количество парков имеет целое значение
data['parks_around_3000'] = data['parks_around_3000'].astype('Int64')

# 6.'ponds_around_3000' float64 преобразуем в формат Int64, количество водоемов имеет целое значение
data['ponds_around_3000'] = data['ponds_around_3000'].astype('Int64')

# 8.Проверим наличие дубликатов - отсутствуют
#display(data.duplicated().sum())
#data.info()

#### 4. Внесение в таблицу расчетных значений<a id='5-bullet'></a>

In [17]:
# 1.Цена квадратного метра
data['price_meter'] = round(data['last_price'] / data['total_area'], 2)

# 2.День недели публикации
data['weekday'] = data['first_day_exposition'].dt.weekday

# 3.Месяц публикации
data['month'] = data['first_day_exposition'].dt.month

# 3.Год публикации
data['year'] = data['first_day_exposition'].dt.year

# 4.Этаж недвижимости
data.loc[data['floor'] == 1, 'floor_realty'] = 'ground floor'
data.loc[data['floor'] == data['floors_total'], 'floor_realty'] = 'last floor'
data.loc[(data['floor'] != data['floors_total']) & (data['floor'] != 1), 'floor_realty'] = 'other floor'

# 5.Отношение жилой к общей площади
data['living/total_area'] = round(data['living_area'] / data['total_area'] * 100, 1) 

#6.Отношение площади кухни к общей площади
data['kitchen/total_area'] = round(data['kitchen_area'] / data['total_area'] * 100, 1)

#7.Расчетные значения
(data.loc[:,['floor_realty', 'rooms', 'total_area', 'living_area', 'kitchen_area', 
             'living/total_area', 'kitchen/total_area', 'price_meter', 'weekday', 
             'month', 'year']].head())

,floor_realty,rooms,total_area,living_area,kitchen_area,living/total_area,kitchen/total_area,price_meter,weekday,month,year
0,other floor,3,108.0,51.0,25.0,47.2,23.1,120370.37,3,3,2019
1,ground floor,1,40.4,18.6,11.0,46.0,27.2,82920.79,1,12,2018
2,other floor,2,56.0,34.3,8.3,61.2,14.8,92785.71,3,8,2015
3,other floor,3,159.0,45.0,9.5,28.3,6.0,408176.10,4,7,2015
4,other floor,2,100.0,32.0,41.0,32.0,41.0,100000.00,1,6,2018


## 3. Исследовательский анализ данных
### 1. Изучение параметров: площадь, цена, число комнат, высота потолков<a id='6-bullet'></a>

In [18]:
# Изучим основные параметры квартир: площадь, цена, число комнат, высота потолков
def data_info(df):
    data_stat_info = (df.pivot_table(
        index = 'rooms', 
        values=['last_price', 'total_area', 'floor', 'ceiling_height', 'days_exposition'], 
        aggfunc={'days_exposition': 'count', 'last_price': 'median', 'total_area': 'median',
                 'floor': 'median', 'ceiling_height': 'median'}))

    data_stat_info['index'] = data_stat_info.index

    # Переставляем местами столбцы таблицы для удобства восприятия
    data_stat_info = (data_stat_info[['index', 'days_exposition', 'last_price', 'total_area', 'ceiling_height', 'floor']]
                        .sort_values('days_exposition', ascending=False)
                        .reset_index(drop =True))

    # Переименуем столбцы, 'number_of_ads' - количество квартир
    (data_stat_info.set_axis(['rooms', 'number_of_ads', 'last_price_median', 'total_area_median', 'ceiling_height_median', 'floor_median']
                        , axis = 'columns', inplace = True))
    return data_stat_info

In [19]:
# Расчетные параметры медианных значений
data_stat = data_info(data)
data_stat.head(17)

,rooms,number_of_ads,last_price_median,total_area_median,ceiling_height_median,floor_median
0,1,8238,3582000.0,36.30,2.640,5.0
1,2,7942,4772669.0,53.00,2.600,4.0
2,3,5816,6200000.0,73.00,2.650,4.0
3,4,1181,9000000.0,99.10,2.800,4.0
4,5,326,14000000.0,146.45,3.100,4.0
5,6,105,19000000.0,184.30,3.200,3.0
6,7,59,24900000.0,214.00,3.380,4.0
7,8,12,23095000.0,235.70,3.420,3.0
8,9,8,23750000.0,300.95,3.200,3.5
9,10,3,23000000.0,272.60,3.150,6.0


In [20]:
# Отфильтруем выбросы по параметрам 'rooms', 'last_price', 'total_area', 'ceiling_height', 'days_exposition'
data = data.query('rooms < 8 and 100000 < last_price < 28510000 and 10 < total_area < 250')
data = data.query('2.4 < ceiling_height <= 4.5 and days_exposition < 600')

In [21]:
# Рассмотрим параметры: площадь, цена, число комнат, высота потолков на графиках и внесем корректировки в фильтры
#info(data[['rooms', 'last_price', 'total_area', 'ceiling_height', 'days_exposition']])

#### Вывод по количеству комнат Rooms
* График rooms имеет форму распределения Пуассона, что свидетельствует о том что данные верные, максимальное количество квартир однокомнатные, далее с ростом комнат количество падает вместе со спросом.
* В процессе анализа были отфильтрованы квартиры с числом комнат более 8, т.к. похожи на выбросы.

#### Вывод по цене на момент снятия с публикации last_price
* График last_price имеет форму нормального распределения, что свидетельствует о том что данные соответствуют действительности
* В процессе анализа были отфильтрованы квартиры со стоимостью менее 100 тысяч и более 25 миллионов, такие квартиры встречаются крайне редко и больше похоже на выбросы.

#### Вывод по площади квартир  total_area
* График total_area имеет форму распределения Пуассона, что свидетельствует о том что существует минимальная площадь пригодная к проживанию и она начинается от 30 м2.
* Минимальная площадь 12 м2, средняя площадь 57 м2, стандартное отклонение 24 м2.
* Такое распределение площади отлично характеризует зависимость минимальной/максимальной площади квартиры от достатка покупателей. Основная масса жителей может выделить деньги на жилье площадью от 30 до 75 м2.

#### Вывод по высоте потолков ceiling_height
* График ceiling_height имеет форму распределения Пуассона, что свидетельствует о том что данные соответствуют действительности.
* В процессе анализа были отфильтрованы квартиры с высотой потолков менее 2,4 м и более 4,5 м, такой разброс высоты потолков характеризуется стандартизацией строительства в разные годы.

#### Вывод по времени продажи квартир days_exposition
* Время продажи колеблется от 1 дня до 600 дней.
* Среднее время продажи составляет 137 дней.
* Медиана продаж составляет 87 дней.
* Можно считать что продажи прошли очень быстро, если квартира продалась за 42 дня, что соответствует Q1 первому квартилю.
* Продажа идет необычно долго, если квартира продается более 191 дня что соответствует Q3-третьему квартилю.

### 2. Изучение факторов которые больше всего влияют на стоимость квартиры. 
#### Когда можно считать что продажи прошли очень быстро, когда необычно долго?<a id='7-bullet'></a>

In [22]:
# Напишем функцию для построения диаграмм зависимости цены от параметра
def chart_price(df, tip):
    fig1 = px.scatter(df, x=tip, y="last_price", size="last_price", color="last_price", 
                      log_x=True, title = 'Распределение ' + tip + ' относительно цены квартиры')
    fig1.show()
    print('Коэффициент корреляции Пирсона:', df["last_price"].corr(df[tip]))

In [23]:
# Напишем функцию для построения диаграмм размаха
def relationship1_price(df, tip):
    fig = px.box(df, x=tip, y='last_price', points="all",
                title = 'Распределение ' + tip + ' относительно цены квартиры')
    fig.show()

In [24]:
# Напишем функцию разложения цены относительно квартиля
def describe_prise(df, column):
    data_price = df.groupby(column)['last_price'].describe() / 1000000
    data_price['count'] = (data_price['count'] * 1000000).astype(int)
    round_3 = ['mean', 'std', 'min', '25%', '50%', '75%', 'max']
    data_price[round_3] = round(data_price[round_3], 3)
    display(data_price)

In [25]:
# Выясним, зависит ли цена от площади недвижимости
# Также выясним коэффициент корреляции Пирсона для зависимости цены от площади
#chart_price(data, "total_area")

#### Вывод по зависимости цены от квадратных метров недвижимости
* По диаграмме рассеяния хорошо видно что стоимость квартир растет с ростом квадратных метров практически линейно.
* Коэффициент корреляции Пирсона для зависимости цены от площади однозначно говорит что зависимость существует.

In [26]:
# Выясним, зависит ли цена от числа комнат в квартире 
#relationship1_price(data, 'rooms')

In [27]:
describe_prise(data, 'rooms')

,count,mean,std,min,25%,50%,75%,max
rooms,,,,,,,,
1,7870,3.781,1.653,0.430,2.900,3.570,4.400,28.000
2,7527,5.430,3.125,0.430,3.690,4.750,6.500,28.000
3,5349,7.292,4.268,0.550,4.700,6.100,8.650,28.100
4,1000,10.056,5.819,0.999,5.850,8.500,13.000,28.178
5,222,12.675,5.566,1.650,9.000,12.425,15.468,28.000
6,62,15.539,4.853,4.390,12.125,15.295,19.375,26.000
7,30,18.438,4.891,7.900,15.000,17.150,22.788,27.500


#### Вывод по зависимости цены от числа комнат в квартире
* По диаграмме размаха мы наблюдаем - что стоимость квартир растет с количеством комнат в них, причем стоимость частично пересекается по всей видимости по причине разного качества жилья и близости к центру, если отбросить выбросы то цены начинаются от следующих отметок:
* 1-ые стоят от 0,43 млн.
* 2-ые стоят от 0,43 млн. 
* 3-ые стоят от 0,55 млн. 
* 4-ые стоят от 1 млн.
* 5-ые стоят от 1,65 млн.
* 6-ые стоят от 4,39 млн.
* 7-ые стоят от 7,9 млн.

In [28]:
# Выясним, зависит ли цена от этажа (первого (1), последнего(2) или другого(3)) 
#relationship1_price(data, "floor_realty")

In [29]:
# Проанализируем изменение цены относительно расположения квартир на первом, последнем и другом этажах в млн.
describe_prise(data, 'floor_realty')

,count,mean,std,min,25%,50%,75%,max
floor_realty,,,,,,,,
ground floor,2693,4.498,2.998,0.43,2.799,3.900,5.3,26.000
last floor,3158,5.391,4.140,0.43,2.962,4.224,6.3,28.178
other floor,16211,5.854,3.817,0.45,3.590,4.750,6.8,28.114


#### Вывод по зависимости цены от этажа расположения квартиры
* По диаграмме размаха и ценавому анализу по квартилям хорошо заметен рост цены от первого этажа к последнему и далее этажи между. Мы нашли весомые доказательства того что расположение квартиры в доме влияет на цену.

In [30]:
# Выясним, зависит ли цена от удалённости от центра
# Коэффициент корреляции Пирсона для зависимости цены от удалённости от центра
#chart_price(data, "city_centers_nearest")

#### Вывод по зависимости цены от удаленности от центра
* Анализируя диаграмму рассеяния зависимости можно сделать следующие выводы:
* Минимальная стоимость квартир при удалении от центра падает от 5.8 млн., до 1.8 млн.
* Максимальная стоимость квартир при удалении также падает от 24 млн. до 18 млн.
* Большинство квартир агломерации Санкт-Петербурга продается на удалении от центра, от 10 км. до 37 км.

In [31]:
# Выясним, зависит ли цена от дня недели в который было выставлено объявление
#relationship1_price(data, 'weekday')

In [32]:
# Проанализируем изменение цены относительно дня недели в млн.
describe_prise(data, 'weekday')

,count,mean,std,min,25%,50%,75%,max
weekday,,,,,,,,
0,3362,5.655,3.868,0.43,3.405,4.60,6.500,28.000
1,3896,5.651,3.757,0.43,3.400,4.65,6.600,28.178
2,3687,5.704,3.926,0.45,3.400,4.60,6.600,28.114
3,3956,5.719,3.954,0.45,3.400,4.55,6.650,28.178
4,3771,5.562,3.709,0.45,3.400,4.50,6.500,28.000
5,1811,5.419,3.601,0.55,3.300,4.50,6.450,28.000
6,1579,5.425,3.511,0.47,3.370,4.50,6.288,26.500


#### Вывод по зависимости цены от дня недели
* Анализируя диаграмму размаха зависимости и квартили цены можно сделать следующие выводы:
* Хорошо заметно что по выходным люди предпочитают отдыхать а не заниматься продажами недвижимости, объем продаж падает в среднем на 50%.
* Медианная стоимость квартир незначительно изменятся, по выходным квартиры продают дешевле.

In [33]:
# Выясним, зависит ли цена от месяца в котором было выставлено объявление
#relationship1_price(data, 'month')

In [34]:
# Проанализируем изменение цены относительно месяца в млн.
describe_prise(data, 'month')

,count,mean,std,min,25%,50%,75%,max
month,,,,,,,,
1,1384,5.634,3.912,0.48,3.365,4.545,6.500,27.840
2,2478,5.652,3.890,0.45,3.350,4.600,6.552,28.178
3,2369,5.607,3.733,0.55,3.400,4.600,6.500,28.000
4,2249,5.739,3.779,0.43,3.500,4.750,6.700,28.000
5,1158,5.607,3.764,0.60,3.440,4.500,6.600,28.050
6,1618,5.366,3.696,0.43,3.250,4.310,6.350,27.420
7,1577,5.680,3.899,0.44,3.400,4.600,6.700,28.114
8,1650,5.585,3.714,0.45,3.391,4.500,6.500,27.850
9,1858,5.734,3.874,0.52,3.441,4.600,6.600,28.178


#### Вывод по зависимости цены от месяца
* Анализируя диаграмму рассеяния зависимости можно сделать следующие выводы:
* Хорошо заметно что в летние месяцы, перед и после нового года объемы рынка недвижимоси падают.
* В целом медианная цена недвижимости колеблется в течении 11 месяцев около среднего значения, но в июне цены минимальны относительно всего года.

In [35]:
# Выясним, зависит ли цена от года в котором было выставлено объявление, исключая 2014 и 2019 года т.к. 
# по ним информация не за все месяцы
#relationship1_price(data, 'year')

In [36]:
# Проанализируем изменение цены относительно года в млн.
describe_prise(data, 'year')

,count,mean,std,min,25%,50%,75%,max
year,,,,,,,,
2014,43,10.921,7.511,1.30,5.410,8.585,15.311,27.934
2015,639,6.401,4.616,0.65,3.620,5.000,7.075,27.000
2016,2278,5.572,3.762,0.58,3.355,4.413,6.500,28.000
2017,7886,5.561,3.807,0.45,3.332,4.489,6.500,28.114
2018,8398,5.490,3.664,0.43,3.350,4.544,6.450,28.178
2019,2818,5.970,3.843,0.43,3.700,5.000,7.000,28.000


#### Вывод по зависимости цены от года <a id='13-bullet'></a>
* Анализируя диаграмму рассеяния зависимости можно сделать следующие выводы:
* Данные подтверждают, что года 2014 и 2019 по количеству сделок неполные - эти данные игнорируем.
* Учитывая что сбор данных начался с 2014 года количество пользователей/сделок растет от года к году.
* Медианная стоимость квартир плавно падает от 2015 года к 2018-му, что может отражать как экономическую ситуацию в стране так и количество предложения на рынке.

#### 4. Анализ стоимости недвижимости 10-ти основных населенных пунктов агломерации<a id='9-bullet'></a>

In [37]:
# 10 населённых пунктов с наибольшим числом объявлений, определим в них медианную цену квадратного метра
top_data= (data.
               pivot_table(index='locality_name', values=['price_meter', 'last_price'], 
               aggfunc=({'last_price': 'count', 'price_meter': 'mean'}))
              .sort_values('last_price', ascending=False)
              .head(10))

# Внесем наименования населенных пунктов в таблицу
top_data['locality_name'] = top_data.index
top_data['price_meter'] = round(top_data['price_meter'], 2)
top_data = top_data.reset_index(drop=True)

# Переименуем столбцы
top_data = top_data[['locality_name', 'last_price', 'price_meter']]
top_data.set_axis(['locality_name', 'quantity', 'price_meter'], axis = 'columns', inplace = True)
top_data = pd.DataFrame(top_data)
display(top_data) 
print()

,locality_name,quantity,price_meter
0,Санкт-Петербург,14491,110286.15
1,поселок Мурино,534,85788.59
2,поселок Шушары,426,78661.24
3,Всеволожск,376,68787.75
4,Пушкин,342,102546.09
5,Колпино,327,75226.59
6,поселок Парголово,318,90006.71
7,Гатчина,288,68981.45
8,деревня Кудрово,276,92706.65
9,Выборг,223,58293.20


In [38]:
# Отфильтруем исходную таблицу по 10 основным населенным пунктам для построения графика
top_data_list =  top_data['locality_name']
data_vilage = data.query('locality_name in @top_data_list and price_meter < 300000')

In [39]:
def relationship2_price(df, tip):
    fig = px.box(df, x=tip, y='price_meter', points="all",
                title = 'Распределение ' + tip + ' относительно цены квартиры')
    fig.show()
#relationship2_price(data_vilage, 'locality_name')

#### Вывод по 10 основным населенным пунктам
* Город с максимальным количеством объявлений в топ 10 Санкт-Петербург имеет максимальную стоимость квадратного метра.
* Город с минимальным количеством объявлений в топ 10 Выборг имеет минимальную стоимость квадратного метра.

#### 5. Исследование на тему: какая область Санкт-Петербурга входит в понятие центр<a id='10-bullet'></a>

In [40]:
# Создадим столбец для квартир в Санкт_Петербурге с расстоянием до центра в км
data['up_to_center'] = round(data[data['locality_name'] == "Санкт-Петербург"]['city_centers_nearest'] / 1000, 0)
# Удалим строку с выбросом по last_price = 14млн. на удалении 27 км от центра, 
# который был обнаружен в процессе построения графика
#display(data_spb.query('up_to_center == 27.0'))
data = data.drop([748]).reset_index(drop=True)

# Создадим функцию, для построения диаграммы рассеяния стоимости от расстояния до центра
def price_km(df):
    # Посчитаем среднюю цену для каждого километра
    df_mean = df.pivot_table(index='up_to_center', values='last_price')
    # Внесем наименования населенных пунктов в таблицу
    df_mean['up_to_center'] = df_mean.index
    df_mean = df_mean.reset_index(drop=True)
    # Выведем диаграмму рассеяния по параметрам цена/расстояние от центра
    fig1 = px.scatter(df_mean, x="last_price", y="up_to_center", size="last_price" , 
                      color="up_to_center",  log_x=True, title = 'График зависимсти цены от расстояния')
    fig1.show()

In [41]:
# График зависимости цены от расстояния от центра
#price_km(data)

#### Вывод
* На диаграмме рассеяния отлично виден разрыв по цене - который делит все квартиры в Санкт-Петербурге в рамках исследования на 2 группы: центр и переферия.
* В группе центр расположены квартиры стоимостью от 8 млн. и на расстоянии от 8.7 км.
* В группе переферия находятся квартиры стоимостью до 6.62 млн. и на расстоянии до 9 км.

#### 6. Анализ сегмента квартир в центре Санкт-Петербурга по параметрам: <a id='11-bullet'></a>
* Площадь
* Цена
* Число комнат 
* Высота потолков 
* Обратим внимание на факторы, которые влияют на стоимость квартиры (число комнат, этаж, удалённость от центра, дата размещения объявления).

In [42]:
# Выделим сегмент квартир в центре Санкт-Петербурга
data_centr = data.query('last_price > 8000000 and city_centers_nearest <= 8700')
data_centr.head()

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,studio,open_plan,kitchen_area,balcony,locality_name,airports_nearest,city_centers_nearest,parks_around_3000,parks_nearest,ponds_around_3000,ponds_nearest,days_exposition,price_meter,weekday,month,year,floor_realty,living/total_area,kitchen/total_area,up_to_center
3,2,10000000.0,100.0,2018-06-19,2,3.03,14,32.0,13,False,False,False,41.0,0,Санкт-Петербург,31856,8098.0,2,112.0,1,48.0,121.0,100000.00,1,6,2018,other floor,32.0,41.0,8.0
46,20,11795000.0,136.0,2017-09-22,6,3.00,2,94.0,2,False,False,False,11.0,1,Санкт-Петербург,16510,6582.0,1,526.0,0,NaN,201.5,86727.94,4,9,2017,last floor,69.1,8.1,7.0
56,2,20000000.0,118.0,2018-09-11,3,3.00,9,68.0,7,False,False,False,16.0,0,Санкт-Петербург,26055,4800.0,1,648.0,1,779.0,37.0,169491.53,1,9,2018,other floor,57.6,13.6,5.0
90,32,9600000.0,90.0,2017-09-26,4,2.80,5,67.0,2,False,False,False,8.0,0,Санкт-Петербург,23277,2818.0,0,NaN,0,NaN,104.0,106666.67,1,9,2017,other floor,74.4,8.9,3.0
97,9,10500000.0,124.1,2017-05-25,5,3.20,4,79.4,3,False,False,False,20.0,0,Санкт-Петербург,26825,5527.0,1,384.0,0,NaN,6.0,84609.19,3,5,2017,other floor,64.0,16.1,6.0


In [43]:
# Рассмотрим параметры: площадь, цена, число комнат, высота потолков на графиках и внесем корректировки в фильтры
#info(data_centr[['rooms', 'last_price', 'total_area', 'ceiling_height', 'days_exposition']])

In [44]:
summary_table = data_info(data_centr)
summary_table

,rooms,number_of_ads,last_price_median,total_area_median,ceiling_height_median,floor_median
0,3,705,11999000.0,94.00,2.84,4.0
1,2,503,10500000.0,71.00,2.80,4.0
2,4,348,12250000.0,112.15,3.00,4.0
3,5,134,13090000.0,141.30,3.10,3.0
4,1,81,10360000.0,58.00,2.75,4.0
5,6,46,16495000.0,166.75,3.20,3.0
6,7,28,17650000.0,187.70,3.38,3.5


### Выводы по характеристикам квартир в центре Санкт-Петербурга
#### total_area - общая площадь
* График total_area квартир в центре имеет близкую к нормальной  форму распределения, что свидетельствует о том что в центре сегмент недвижимости смещен в сторону более комфортабельного жилья в отличи от в целом по агломерации.
* Минимальная площадь 31 м2, медианная площадь 95 м2.
* Такое распределение площади отлично характеризует зависимость минимальной/максимальной площади квартиры от достатка покупателей. Жить в квартире площадью от 76 до 118 м2 комфортно. 

#### last_price - цена продажи квартиры
* График last_price имеет форму распределения Пуассона, что свидетельствует о том что данные верные, т.к. квартиры в этом срезе находятся в центре то стоимость начинается с обозначенного границами центра уровня от 8 млн., далее по приближению к центру стоимсть закономерно растет до в среднем 25 млн., а количество квартир падает.
* Распределение центра отличается от распределения агломерации (Нормальное) т.к. количество квартир от границы центра к середине резко падает с ростом цены, по нескольким причинам. В центре сосредоточены восновном административные и торговые здания - это вызывает рост стоимости жилой недвижимости. Получать прибыль с площади выгоднее чем просто жить на ней.

#### last_price rooms - количество комнат
* График rooms имеет нормальную форму распределения, что свидетельствует о том что данные верные.
* Можно сделать вывод, что люди которые могут позволить себе квартиру в центре стремяться к комфорту и готовы за это платить. Основное количество квартир имеют от 2-х до 4-х комнат.
* В целом по агломерации с распределением Пуассона другая тенденция, людям с низким достатком достаточно минимально возможной площади - это 1-комнатная квартира.

#### ceiling_height - высота потолков
* График ceiling_height высота потолков - стремиться к опреденным стандартным цифрам отражающим строительные нормы. 

In [45]:
# Построим график зависимости цены от расстояния в центре
#price_km(data_centr)

#### Вывод по зависимости цены от удаления от центра
* На диграмме хоршо виден выброс на 7 км., в остальном диаграмма демонстрируют рост количества квартир от центра к переферии и рост стоимости от переферии к центру, что подтверждается тендецией в целом по агломерации.

In [46]:
#relationship1_price(data_centr, 'rooms')

In [47]:
describe_prise(data_centr, 'rooms')

,count,mean,std,min,25%,50%,75%,max
rooms,,,,,,,,
1,81,11.825,4.253,8.010,9.000,10.360,13.400,28.000
2,503,12.151,4.400,8.002,9.000,10.500,13.990,28.000
3,705,13.417,4.863,8.100,9.500,11.999,15.900,28.000
4,348,13.998,5.374,8.100,9.700,12.250,16.828,28.178
5,134,14.395,4.568,8.260,11.000,13.090,16.372,28.000
6,46,16.569,4.382,9.100,12.992,16.495,19.500,26.000
7,28,18.892,4.604,12.000,15.000,17.650,22.925,27.500


#### Вывод по зависимости цены от числа комнат в квартире в центре
* По диаграмме рассеяния мы наблюдаем - что стоимость всех квартир кроме 6-и комнатных и выше начинается от 8 млн. независимо от количества комнат. 
* Можно заметить что в топовом сегменте стоимость квартиры уже слабо коррелирует с количеством комнат, на первый план выходят другие параметры которые нужно рассматривать по каждой квартире индивидуально.

In [48]:
# Диграммы распределния зависимости цены от этажа
#relationship1_price(data_centr, "floor_realty")

In [49]:
describe_prise(data_centr, 'floor_realty')

,count,mean,std,min,25%,50%,75%,max
floor_realty,,,,,,,,
ground floor,115,12.313,4.422,8.100,9.2,10.50,13.326,26.000
last floor,313,13.408,5.019,8.100,9.4,11.95,15.400,28.178
other floor,1417,13.414,4.927,8.002,9.5,11.94,15.900,28.114


#### Вывод по зависимости цены от этажа расположения квартиры в центре
* На первом этаже квартиры стоят дешевле чем на последнем и "других" этажах, жить с видом на проезжую часть удовольствие сомнительное.
* На последнем и "других" этажах стоимость отличается незначительно.

In [50]:
# Диграмма распределния цены от даты размещения объявления
# fig1 = px.scatter(data_centr, x="last_price", y="first_day_exposition", size="last_price" , color="last_price",  log_x=True)
# fig1.show()

#### Вывод по зависимости цены от даты размещения
* Анализируя диаграмму рассеяния зависимости можно сделать аналогичные выводы как и в целом по агломерации:
* С каждым годом начиная с 2016 растут все сегменты рынка - возможно это также связано с ростом популярности сервиса, но заметного тренда на увеличение стоимости квартир не наблюдается.

#### Общий вывод по проведенному исследованию <a id='12-bullet'></a>
* [Основные параметры квартир (площадь, цена, число комнат, этаж, высота потолков) в агломерасции Санкт-Петербурга и его центра подчиняются распределению Пуассона и нормальному распредлению, что подтверждает достоверность данных, а также позволит используя данные исследования отсеивать неадекватную стоимость в объявлениях.](#6-bullet)
* [Можно считать что продажи прошли очень быстро, если квартира продалась за 45 дня, что соответствует Q1 первому квартилю. Продажа идет необычно долго, если квартира продается более 172 дня что соответствует Q3-третьему квартилю.](#7-bullet)
* [Основное влияние на цены оказывает экономическая ситуация в стране и предложение на рынке т.к. за время исследования доходы граждан не увеличились тенденций к росту цен на недвижимость не обнаружено.](#13-bullet)
* [Центр города и переферия разделены разным достатком и потребностями покупателей, что выявлено в пункте 3,6 исследования.](#11-bullet)  